In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  

import pandas as pd
import numpy as np

from datetime import datetime, timedelta
import parseIntervalFiles as pif
import parseActivityFiles as paf
#pun intended :)
import consolidateFiles as cf
import datacleaning as cl

## ! Parse activity files and parse interval files are to be replaced with the corresponding database queries as soon as they are available 

## 0 - Pipeline configuration 
* set the input/output directories, user id and verbose level

In [2]:
verbose = True
    
DATA_PATH = "C:\\Users\\ju\\GDrive\\Projects\\HeRV\\Data\\" 
# DATA_PATH = "/home/ju/GDrive/Projects/HeRV/Data/"
    
RAW_PATH = DATA_PATH + "Raw"
PRE_PATH = DATA_PATH + "PreProcessed"     

## 1 - Extract sessions from raw data (intervals + annotations files)

### 1.1 - List activities with session start/stop datetime
Parses activities files and prints all errors found (activities without start or stop, invalid dates, etc)

In [3]:
sessions = []
for user in range(7):
    print ("------- parsing activity files for user ", user, " -------")
    user_sess = paf.get_user_sessions(user, dirname=RAW_PATH)
    print ("------- parsing interval files for user ", user, " -------")
    user_sess = cf.sessions_add_beats(sessions=user_sess, dirname=RAW_PATH+"\\"+str(user), verbose=False)
    sessions.extend(user_sess)

------- parsing activity files for user  0  -------
reading act170929.csv ... 
reading act170930.csv ... 
reading act171001.csv ... 
reading act171003.csv ... 
reading act171004.csv ... 
reading act171005.csv ... 
orphan start in: ['2017-10-05 12:54:00', 'start', 'eat', 'sitting', '']
orphan start in: ['2017-10-05 13:48:39', 'start', 'rest-active', 'sitting', '']
orphan start in: ['2017-10-05 13:57:37', 'start', 'movement', 'standing', '']
orphan start in: ['2017-10-05 17:55:44', 'start', 'focused-active', 'sitting', '']
orphan start in: ['2017-10-05 21:53:45', 'start', 'rest-active', 'sitting', '']
orphan start in: ['2017-10-05 23:47:45', 'start', 'focused-active', 'sitting', 'sitting on bed, installing and using pandas']
reading act171006.csv ... 
reading act171010.csv ... 
orphan start in: ['2017-10-10 23:23:37', 'start', 'leisure-passive', 'sitting']
reading act171011.csv ... 
orphan stop in: ['2017-10-11 0:04:41', 'stop', '', '', '']
orphan start in: ['2017-10-11 12:30:10', 'start

In [4]:
df = pd.DataFrame(sessions)

### Include column with beats count for ease of use 

In [5]:
df['beatscount'] = df['rr'].apply(len)

In [6]:
df.sample(5)

,activity,duration,notes,posture,rr,start,stop,user,beatscount
433,rest-active,754,NaN,stand,"[{'date': 2017-11-12 16:08:07, 'interval': 686...",2017-11-12 16:08:06,2017-11-12 16:20:40,2,647
128,rest-passive,1282,bus,stand,"[{'date': 2017-11-01 07:40:52, 'interval': 428...",2017-11-01 07:40:51,2017-11-01 08:02:13,0,2616
21,rest-active,1299,playing,sitting,"[{'date': 2017-09-30 11:53:01, 'interval': 745...",2017-09-30 11:53:00,2017-09-30 12:14:39,0,1353
85,household-chores,3093,,standing,"[{'date': 2017-10-14 13:16:59, 'interval': 644...",2017-10-14 13:16:58,2017-10-14 14:08:31,0,4959
162,focused-active,1605,NaN,sit,"[{'date': 2017-11-09 13:40:43, 'interval': 705...",2017-11-09 13:40:41,2017-11-09 14:07:26,0,2342


## 2 - Cleaning data

### 2.1 Removing outliers and artifacts

(Note: For now, beats won't be saved to the output file, so, when loading the sessions file and getting
beats from DB again, for example for fragmentation, they will have to be removed again. But this is a reasonably low-cost op)

In [7]:
df['rr'] = df['rr'].apply(cl.clean_rr_series)

In [8]:
df['beatscount_clean'] = df['rr'].apply(len)

In [9]:
df['removed_artifacts'] = df['beatscount'] - df['beatscount_clean']
df['beatscount'] = df['beatscount_clean']
df = df.drop(['beatscount_clean'], axis=1)

In [10]:
df.describe()

,duration,user,beatscount,removed_artifacts
count,498.000000,498.000000,498.000000,498.000000
mean,3163.447791,1.226908,3573.600402,21.955823
std,6632.700097,1.570195,6291.462270,61.224018
min,0.000000,0.000000,0.000000,0.000000
25%,840.000000,0.000000,925.000000,0.000000
50%,1325.500000,1.000000,1642.000000,1.000000
75%,2417.000000,2.000000,3504.500000,12.000000
max,80732.000000,6.000000,46856.000000,591.000000


### 2.2 Removing sessions without beats recorded or with too few of them

In [11]:
l = len(df)
df = df[df['beatscount'] > 100]
print(l - len(df), ' sessions out of', l, 'removed for lack of interval data')

44  sessions out of 498 removed for lack of interval data


### 2.3 Removing wrong activities and grouping the ones with too few examples

In [12]:
df.groupby('activity').count()['duration']

activity
eat                 52
exercise-high       17
exercise-low         2
focused-active      96
focused-active       1
focused-passive     43
focused-passive      1
household-chores    28
leisure-active       7
leisure-passive     46
movement            55
not-recorded         4
rest-active         27
rest-passive        22
sleep               53
Name: duration, dtype: int64

In [13]:
# we have too few exercise sessions to classify intensity
df.at[df['activity']=='exercise-low', 'activity'] = 'exercise'
df.at[df['activity']=='exercise-high', 'activity'] = 'exercise'

# also we have too few leisure active sessions
df.at[df['activity']=='leisure-active', 'activity'] = 'leisure'
df.at[df['activity']=='leisure-passive', 'activity'] = 'leisure'

# these were just mispelled
df.at[df['activity']=='focused-passive ', 'activity'] = 'focused-passive'
df.at[df['activity']=='focused-active ', 'activity'] = 'focused-active'

df = df[df['activity']!='not-recorded']

In [14]:
df.groupby('activity').count()['duration']

activity
eat                 52
exercise            19
focused-active      97
focused-passive     44
household-chores    28
leisure             53
movement            55
rest-active         27
rest-passive        22
sleep               53
Name: duration, dtype: int64

## 3 - Aggregating data

In [15]:
dic = df.to_dict(orient='records')
for i in dic:
    i.update(cf.features_from_dic(i['rr']))
print(dic[0]['rmssd'])

239.316870462


In [16]:
df = pd.DataFrame(dic)
df.sample(3)

,activity,beatscount,duration,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,...,posture,removed_artifacts,rmssd,rr,sdnn,start,stop,total_power,user,vlf
68,sleep,24195,23722,15586.319714,82.363317,3337.541489,0.214133,17.636683,62.031353,1001.514280,...,lying down,28,195.252105,"[{'date': 2017-10-14 04:05:26, 'interval': 739...",186.490859,2017-10-14 04:05:25,2017-10-14 10:40:47,22130.930505,0,3207.069302
317,focused-passive,410,900,85.214999,14.568042,499.729758,5.864340,85.431958,94.003603,640.339024,...,sit,0,14.469173,"[{'date': 2017-10-06 17:00:01, 'interval': 608...",37.289913,2017-10-06 17:00:00,2017-10-06 17:15:00,887.822949,2,302.878192
132,sleep,33911,27396,6289.079825,73.953427,2215.028898,0.352202,26.046573,75.106028,822.886409,...,lie,194,114.949791,"[{'date': 2017-11-09 00:24:07, 'interval': 711...",147.958598,2017-11-09 00:24:02,2017-11-09 08:00:38,10482.081037,0,1977.972314


In [17]:
df['sess_id'] = df.index
df_export = df.drop(['rr'], axis = 1)
df_export.sample(1)

,activity,beatscount,duration,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,...,posture,removed_artifacts,rmssd,sdnn,start,stop,total_power,user,vlf,sess_id
225,focused-active,4505,3638,915.742931,35.593396,1657.045918,1.80951,64.406604,73.263658,826.648169,...,sit,0,51.726611,78.475694,2017-10-30 13:24:45,2017-10-30 14:25:23,3911.237346,1,1338.448497,225


In [18]:
df_export.describe()

,beatscount,duration,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,nn50,pnn50,removed_artifacts,rmssd,sdnn,total_power,user,vlf,sess_id
count,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000
mean,3925.582222,3126.822222,2067.095548,41.767744,1693.621250,1.955210,58.232256,87.261169,730.931761,932.093333,17.414980,24.200000,65.040535,91.059682,5138.145294,1.280000,1377.428496,224.500000
std,6502.564990,5742.013922,3777.616059,17.165852,1691.491027,1.608758,17.165852,19.028288,142.556068,2628.950661,15.433927,63.972711,50.889449,43.606087,6162.382931,1.592521,1240.472581,130.048068
min,105.000000,88.000000,13.635122,8.241489,36.995074,0.172079,14.681505,54.190501,369.893825,0.000000,0.000000,0.000000,5.402758,21.083352,150.076646,0.000000,52.162541,0.000000
25%,1112.000000,900.000000,370.393323,28.610812,673.845680,0.818736,45.016759,74.684912,642.513222,82.000000,5.880139,0.000000,32.713546,61.912978,1745.794584,0.000000,570.739610,112.250000
50%,1789.000000,1428.000000,719.166158,39.474339,1216.947907,1.533292,60.525661,83.176087,733.546638,242.500000,13.670047,1.000000,48.330213,78.514148,3205.454764,1.000000,1022.983694,224.500000
75%,3771.750000,2458.000000,1606.190331,54.983241,1974.461539,2.495209,71.389188,95.142772,817.104119,530.500000,24.888292,14.000000,76.322143,108.261929,5268.507993,2.000000,1713.661655,336.750000
max,46856.000000,34360.000000,22414.537885,85.318495,12243.363436,11.133730,91.758511,164.397688,1144.581425,20895.000000,81.780538,591.000000,290.138931,253.787250,36726.950587,6.000000,7350.357547,449.000000


In [19]:
df_export.to_excel(PRE_PATH + "\\sessions.xlsx")